## Lets extract chords from a song

In [560]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
from functools import reduce
import time
import random
import jl_io as io
import os
import uuid

In [561]:
url = 'https://tabs.ultimate-guitar.com/tab/mariah-carey/all-i-want-for-christmas-is-you-chords-903566'

In [574]:
class ChordExtractor:

    def __init__(self, raw_html_output_directory):
        self.raw_html_output_directory = raw_html_output_directory
        self.driver = self.create_chrome_driver()
        self.first_time = True
        
        if not os.path.isdir(self.raw_html_output_directory):
            os.mkdir(self.raw_html_output_directory)
            
    def extract_song_data(self,url):
        chords_spans = self.get_chord_spans(url)
        
        chords = [span.decode_contents() for span in spans]
        
        song_uuid = str(uuid.uuid4())
        with open(f"{self.raw_html_output_directory}/{song_uuid}.html", "w") as file: # De los datos,como del cerdo, se guarda todo.
            file.write(self.driver.page_source )
    
        info = {
            "url":url,
            "chords":chords,
            "uuid":song_uuid
        }
        
        return info
    
    def get_chord_spans(self,url):
        self.driver.get(url)
        
        if self.driver.page_source == '<html><head></head><body></body></html>':
            raise Exception('Denegation error')

        if self.first_time:
            self.click_on_accept_cookies()
            self.first_time = False

        soup = BeautifulSoup(self.driver.page_source, 'lxml')

        article = soup.findAll('article')[3];
        
        return article.findAll('span', {"style":"color: rgb(0, 0, 0);"})
    
    def click_on_accept_cookies(self):
        try:
            button = self.driver.find_element_by_xpath('//button[contains(text(), "thanks")]')

            button.click()
        except:
            print('cookies banner not found. Ignored')
            
    
    def create_chrome_driver(self):
        options = webdriver.ChromeOptions()
        options.add_argument('--ignore-certificate-errors')
        options.add_argument('--incognito')
        options.add_argument('--headless')
        
        driver = webdriver.Chrome("./chromedriver", options=options)
        return driver

In [575]:
extractor = ChordExtractor('../data/pages/')


In [576]:
data = extractor.extract_song_data(url)
data

{'url': 'https://tabs.ultimate-guitar.com/tab/mariah-carey/all-i-want-for-christmas-is-you-chords-903566',
 'chords': ['G',
  'Em',
  'C',
  'D',
  'G',
  'G/B',
  'C',
  'Cm/Eb',
  'G/B',
  'B7',
  'Em',
  'Cm/Eb',
  'G/D',
  'E7',
  'Am7',
  'Cm6/D',
  'G',
  'Em',
  'C',
  'D',
  'G',
  'C',
  'Eb6',
  'G',
  'G',
  'C',
  'Eb6',
  'G',
  'B7',
  'Em',
  'Eb6',
  'G/D',
  'E7',
  'Am7',
  'Cm6/D',
  'G',
  'Em',
  'C',
  'D',
  'G',
  'G',
  'C',
  'Eb6',
  'G',
  'C',
  'Eb6',
  'G',
  'B7',
  'Em',
  'Eb6',
  'G/D',
  'E7',
  'Am7',
  'Cm6/Eb',
  'G',
  'Em',
  'C',
  'D',
  'B7',
  'Em',
  'B7',
  'Em',
  'Eb6',
  'G/D',
  'E7',
  'Am7',
  'D',
  'G',
  'C',
  'Eb6',
  'G',
  'B7',
  'Em',
  'Eb6',
  'G/D',
  'E7',
  'Am7',
  'Cm6/D',
  'G',
  'Em',
  'C',
  'D',
  'G',
  'Em',
  'C',
  'D',
  'G',
  'Em',
  'C',
  'D'],
 'uuid': '1d9d8b77-c55f-4bfb-b7cb-386993d7fbe1'}

In [580]:
! ls ../data/pages/

1d9d8b77-c55f-4bfb-b7cb-386993d7fbe1.html
52ab9744-4788-4c19-8b71-085b3a67e4fe.html


## Let's prepare a place to save our data 

In [601]:
import pandas as pd
import os.path

class SongData:
    SEPARATOR = '^'
    
    def __init__(self, initial_data_path=None):
        self.df = pd.DataFrame(data=[], columns=['url','name','genre','decade','chords','uuid'])
        
        if initial_data_path is not None and os.path.isfile(initial_data_path):
            self.df = pd.read_csv(initial_data_path, sep=self.SEPARATOR)
            
    def add_basic_data(self,basic_data):
        self.df = self.df.append(basic_data,ignore_index=True)

    def add_details(self,details):
        self.df.loc[self.df['url'] == details["url"], ["chords"]] = str(details["chords"])
        self.df.loc[self.df['url'] == details["url"], ["uuid"]] = details["uuid"]
        
    def has_basic_data(self,url):
        return (self.df['url'] == url).any()
    
    def has_details(self,url):
        return ((self.df['url'] == url) & (self.df['chords'].notnull())).any()
        
    def get_chords(self,url):
        return eval( self.df[self.df['url'] == url]['chords'][0])
    
    def has_genre_and_decade(self, genre, decade):
        return ((self.df['genre'] == genre) & (self.df['decade'] == decade)).any()
    
    def save(self, path):
        self.df.to_csv(path,index=False,sep=self.SEPARATOR)
    

        

In [602]:
data = SongData()
data.df

,url,name,genre,decade,chords,uuid


In [603]:
data.has_basic_data('http.://www.help.com')

False

In [604]:
data.has_details('http.://www.help.com')

False

In [605]:
data.has_genre_and_decade('pop','1980s')

False

In [607]:
data.add_basic_data({
    "name":"Help",
    "url":"http.://www.help.com",
    "genre": "pop",
    "decade": "1980s",
})

In [608]:
data.df

,url,name,genre,decade,chords,uuid
0,http.://www.help.com,Help,pop,1980s,NaN,NaN


In [609]:
data.has_basic_data('http.://www.help.com')

True

In [610]:
data.has_details('http.://www.help.com')

False

In [611]:
data.has_genre_and_decade('pop','1980s')

True

In [589]:
data.add_details({
    "url":"http.://www.help.com",
    "chords": ["E", "F", "G"],
    'uuid': "7656-i87-gj"
})

In [590]:
data.df

,url,name,genre,decade,chords,uuid
0,http.://www.help.com,Help,pop,1980s,"['E', 'F', 'G']",7656-i87-gj


In [591]:
data.has_basic_data('http.://www.help.com')

True

In [592]:
data.has_details('http.://www.help.com')

True

In [593]:
data.get_chords('http.://www.help.com')[0]

'E'

In [594]:
data.save('borrare.csv')

In [595]:
data = SongData('borrare.csv')

In [596]:
data.get_chords('http.://www.help.com')[0]

'E'

In [597]:
! rm borrare.csv

## Link extractor (adapted from previous notebooks)

Find song links for a genre and decade

In [665]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
from functools import reduce

class LinkExtractor:
    BASE_URL = "https://www.ultimate-guitar.com/explore?&type[]=Chords";
    
    def __init__(self):
        self.driver = self.create_chrome_driver()
        self.first_time = True
    
    def get_all_songs(self, genre, decade, first_page, last_page):
        links = self.get_all_filter_song_links(genre['pattern'],decade['pattern'])
        result = [ self.link_to_song_dict(link,genre,decade) for link in links]
        
        return result
    
    def link_to_song_dict(self,link, genre, decade):
        return {
            "name": link.contents[0],
            "url": link['href'],
            "genre": genre["name"],
            "decade": decade["name"]
        }
    
    def get_all_filter_song_links(self,genreFilter,decadeFilter, first_page, last_page):
        list_of_list = [self.get_links_single_page(genreFilter, decadeFilter, f'&page={page}')
                  for page in range(first_page,last_page + 1)]
        
        return reduce(lambda list1, list2: [*list1, *list2], list_of_list)
    
    def get_links_single_page(self,genreFilter,decadeFilter, pageFilter):
        self.driver.get(f'{self.BASE_URL}{genreFilter}{decadeFilter}{pageFilter}')
        
        if self.driver.page_source == '<html><head></head><body></body></html>':
            raise Exception('Denegation error')

        if self.first_time:
            self.click_on_accept_cookies()
            self.first_time = False

        soup = BeautifulSoup(self.driver.page_source, 'lxml')

        return soup.findAll('a', {"class":"_2KJtL _1mes3 kWOod"})
    
    def click_on_accept_cookies(self):
        try:
            button = self.driver.find_element_by_xpath('//button[contains(text(), "thanks")]')

            button.click()
        except:
            print('cookies banner not found. Ignored')
            
    
    def create_chrome_driver(self):
        options = webdriver.ChromeOptions()
        options.add_argument('--ignore-certificate-errors')
        options.add_argument('--incognito')
        options.add_argument('--headless')
        
        driver = webdriver.Chrome("./chromedriver", options=options)
        return driver

## Simplified extractor (adapted from previous notebooks)

We are going to iterate through genres and decades (we will forget about styles)

In [35]:
import jl_io as io
genres = io.from_file('../data/genres')
decades = io.from_file('../data/decades')

In [37]:
potential_number_of_songs = len(genres) * len(decades) * 500
potential_number_of_songs

73500

In [666]:
import jl_io as io

class SimplifiedCombinationsSongExtractor:
    def __init__(self, linkExtractor, chordExtractor, genres, decades, song_data):
        self.linkExtractor = linkExtractor
        self.chordExtractor = chordExtractor
        self.genres = genres
        self.decades = decades
        self.song_data = song_data
        
        self.combinations = []

        for decade in self.decades:
            for genre in self.genres:
                    self.combinations.append( {"decade": decade, "genre": genre})
        
    def extract(self, first_page, last_page):      
        startIndex = 1
        
        return self.extract_from(startIndex)
            
    def extract_from(self, startIndexBase1,first_page, last_page):    
        for index, combination in enumerate(self.combinations[startIndexBase1 - 1:]):
            new_extracted_songs = []
            genre = combination["genre"]
            decade = combination["decade"]
            
            if song_data.has_genre_and_decade(genre["name"], decade["name"]):
                print(f'{genre["name"]},{decade["name"]} already extracted')
                continue
            
            song_basic_data_array = []          
            try:
                song_basic_data_array = self.linkExtractor.get_all_songs(genre,decade, first_page, last_page)
                      
                for basic_data in song_basic_data_array:
                    self.song_data.add_basic_data(basic_data)
                      
                new_extracted_songs = [self.extract_song_data(index,song) for index,song in enumerate(new_extracted_songs)]
            except Exception as e: 
                print(f'Error in ({genre["name"]},{decade["name"]})')
                print(e)
            
            for index,song in enumerate(song_basic_data_array):
                self.extract_song_data(index,song)
            
            number_of_songs = len(self.song_data.df)
            print(f'Extracted {index+startIndexBase1} of {len(self.combinations)} ({genre["name"]},{decade["name"]}). {number_of_songs} in total')  
        
    def extract_song_data(self,index,song):       
        try:        
            song_details = self.chordExtractor.extract_song_data(song["url"])           
            self.song_data.add_details(song_details)
            print(f'Extracted data from song {index}. "{song["name"]}"')
        except Exception as e: 
            print(f'Error in "{song["name"]}"')
            print(e)


## Let's use it! 

In [612]:
linkExtractor = LinkExtractor()

In [614]:
raw_html_output = '../data/raw_html'

In [615]:
chordExtractor = ChordExtractor(raw_html_output)

In [617]:
song_data = SongData()

In [621]:
extractor = SimplifiedCombinationsSongExtractor(linkExtractor,chordExtractor,genres,decades,song_data )

In [622]:
extractor.extract()

Extracted data from song 0. "All I Want"
Extracted data from song 1. "Take Me To Church (ver 2)"
Extracted data from song 2. "Radioactive"
Extracted data from song 3. "Cigarette Daydreams"
Extracted data from song 4. "Say Something (ver 3)"
Extracted data from song 5. "Counting Stars"
Extracted data from song 6. "Believer (ver 2)"
Extracted data from song 7. "Million Reasons"
Extracted data from song 8. "Sign Of The Times"
Extracted data from song 9. "Demons"
Extracted data from song 10. "Little Talks"
Extracted data from song 11. "High Hopes (ver 2)"
Extracted data from song 12. "Say Something (ver 2)"
Extracted data from song 13. "High Hopes"
Extracted data from song 14. "Demons (ver 2)"
Extracted data from song 15. "Another Love"
Extracted data from song 16. "Take Me To Church"
Extracted data from song 17. "Pompeii (ver 3)"
Extracted data from song 18. "Everglow"
Extracted data from song 19. "Breezeblocks (ver 2)"
Extracted data from song 20. "Somebody Else"
Extracted data from song

Extracted data from song 172. "Bad Liar"
Extracted data from song 173. "Sweet Creature"
Extracted data from song 174. "Chlorine"
Extracted data from song 175. "Youngblood"
Extracted data from song 176. "We Are Young (ver 2)"
Extracted data from song 177. "Youth"
Extracted data from song 178. "Colder Weather"
Extracted data from song 179. "Shake Up Christmas"
Extracted data from song 180. "Little Black Submarines (ver 2)"
Extracted data from song 181. "Tonight You Belong To Me"
Extracted data from song 182. "Dirty Paws"
Extracted data from song 183. "Still Into You (ver 5)"
Extracted data from song 184. "All I Want (ver 6)"
Extracted data from song 185. "December"
Extracted data from song 186. "Blue Aint Your Color"
Extracted data from song 187. "Pumped Up Kicks"
Extracted data from song 188. "Shut Up And Dance"
Extracted data from song 189. "I Wanna Be Yours"
Extracted data from song 190. "A Sky Full Of Stars (ver 2)"
Extracted data from song 191. "The Less I Know The Better (ver 2)"
E

Extracted data from song 342. "Amnesia"
Extracted data from song 343. "Renegades (ver 2)"
Extracted data from song 344. "Unsteady"
Extracted data from song 345. "Two Ghosts"
Extracted data from song 346. "If I Could Fly (ver 4)"
Extracted data from song 347. "Paradise"
Extracted data from song 348. "In The Blood"
Extracted data from song 349. "The Messenger"
Extracted data from song 350. "All I Want"
Extracted data from song 351. "Take Me To Church (ver 2)"
Extracted data from song 352. "Radioactive"
Extracted data from song 353. "Cigarette Daydreams"
Extracted data from song 354. "Say Something (ver 3)"
Extracted data from song 355. "Counting Stars"
Extracted data from song 356. "Believer (ver 2)"
Extracted data from song 357. "Million Reasons"
Extracted data from song 358. "Sign Of The Times"
Extracted data from song 359. "Demons"
Extracted data from song 360. "Little Talks"
Extracted data from song 361. "High Hopes (ver 2)"
Extracted data from song 362. "Say Something (ver 2)"
Extra

Extracted data from song 512. "Say Something (ver 2)"
Extracted data from song 513. "High Hopes"
Extracted data from song 514. "Demons (ver 2)"
Extracted data from song 515. "Another Love"
Extracted data from song 516. "Take Me To Church"
Extracted data from song 517. "Pompeii (ver 3)"
Extracted data from song 518. "Everglow"
Extracted data from song 519. "Breezeblocks (ver 2)"
Extracted data from song 520. "Somebody Else"
Extracted data from song 521. "Death Of A Bachelor"
Extracted data from song 522. "Bad Liar"
Extracted data from song 523. "Sweet Creature"
Extracted data from song 524. "Chlorine"
Extracted data from song 525. "Youngblood"
Extracted data from song 526. "We Are Young (ver 2)"
Extracted data from song 527. "Youth"
Extracted data from song 528. "Colder Weather"
Extracted data from song 529. "Shake Up Christmas"
Extracted data from song 530. "Little Black Submarines (ver 2)"
Extracted data from song 531. "Tonight You Belong To Me"
Extracted data from song 532. "Dirty Pa

Extracted data from song 682. "Dirty Paws"
Extracted data from song 683. "Still Into You (ver 5)"
Extracted data from song 684. "All I Want (ver 6)"
Extracted data from song 685. "December"
Extracted data from song 686. "Blue Aint Your Color"
Extracted data from song 687. "Pumped Up Kicks"
Extracted data from song 688. "Shut Up And Dance"
Extracted data from song 689. "I Wanna Be Yours"
Extracted data from song 690. "A Sky Full Of Stars (ver 2)"
Extracted data from song 691. "The Less I Know The Better (ver 2)"
Extracted data from song 692. "Amnesia"
Extracted data from song 693. "Renegades (ver 2)"
Extracted data from song 694. "Unsteady"
Extracted data from song 695. "Two Ghosts"
Extracted data from song 696. "If I Could Fly (ver 4)"
Extracted data from song 697. "Paradise"
Extracted data from song 698. "In The Blood"
Extracted data from song 699. "The Messenger"
Extracted data from song 700. "All I Want"
Extracted data from song 701. "Take Me To Church (ver 2)"
Extracted data from s

...

Error in "Vanilla Twilight (ver 2)"
Denegation error
Error in "Feel Good Inc (ver 5)"
Denegation error
Error in "Clint Eastwood"
Denegation error
Error in "Crawling (ver 4)"
Denegation error
Error in "Speechless (ver 2)"
Denegation error
Error in "Oo"
Denegation error
Error in "Knives Out"
Denegation error
Error in "Bulletproof (ver 2)"
Denegation error
Error in "Kids (ver 2)"
Denegation error
Error in "In A Manner Of Speaking"
Denegation error
Error in "Animal (ver 4)"
Denegation error
Error in "One More Time"
Denegation error
Error in "Numb"
Denegation error
Error in "How He Loves"
Denegation error
Error in "Paparazzi (ver 2)"
Denegation error
Error in "To Build A Home"
Denegation error
Error in "Crawling (ver 3)"
Denegation error
Error in "What Ive Done (ver 3)"
Denegation error
Error in "In The End"
Denegation error
Error in "Feel Good Inc (ver 4)"
Denegation error
Error in "Electric Feel (ver 3)"
Denegation error
Error in "How To Disappear Completely"
Denegation error
Error in "Fe

Error in "Feel Good Inc (ver 2)"
Denegation error
Error in "Hide And Seek"
Denegation error
Error in "In The End (ver 3)"
Denegation error
Error in "Clint Eastwood (ver 2)"
Denegation error
Error in "Fuck You"
Denegation error
Error in "What Ive Done"
Denegation error
Error in "We Are The People (ver 3)"
Denegation error
Error in "Something About Us"
Denegation error
Error in "Time To Pretend"
Denegation error
Error in "How He Loves (ver 3)"
Denegation error
Error in "The Fear (ver 6)"
Denegation error
Error in "Poker Face (ver 6)"
Denegation error
Error in "We Are The People"
Denegation error
Error in "Not Fair"
Denegation error
Error in "Pyramid Song (ver 3)"
Denegation error
Error in "Speechless (ver 5)"
Denegation error
Error in "Vanilla Twilight (ver 2)"
Denegation error
Error in "Feel Good Inc (ver 5)"
Denegation error
Error in "Clint Eastwood"
Denegation error
Error in "Crawling (ver 4)"
Denegation error
Error in "Speechless (ver 2)"
Denegation error
Error in "Oo"
Denegation err

Error in "Crazy (ver 2)"
Denegation error
Error in "You Know Im No Good (ver 4)"
Denegation error
Error in "Put Your Records On (ver 3)"
Denegation error
Error in "Chasing Pavements (ver 9)"
Denegation error
Error in "Fallin"
Denegation error
Error in "Hurt"
Denegation error
Error in "Love Is A Losing Game"
Denegation error
Error in "You Know Im No Good (ver 2)"
Denegation error
Error in "Chasing Pavements (ver 6)"
Denegation error
Error in "Back To Black (ver 4)"
Denegation error
Error in "Rehab"
Denegation error
Error in "Ordinary People"
Denegation error
Error in "Make You Feel My Love (ver 7)"
Denegation error
Error in "Broken Strings (ver 2)"
Denegation error
Error in "If I Aint Got You"
Denegation error
Error in "Tears Dry On Their Own"
Denegation error
Error in "Chasing Pavements"
Denegation error
Error in "You Give Me Something"
Denegation error
Error in "Make You Feel My Love (ver 9)"
Denegation error
Error in "Make You Feel My Love (ver 5)"
Denegation error
Error in "Hometown

Error in "Make You Feel My Love (ver 2)"
Denegation error
Error in "Sideways (ver 3)"
Denegation error
Error in "Like A Star (ver 5)"
Denegation error
Error in "Crazy (ver 3)"
Denegation error
Error in "Like A Star"
Denegation error
Error in "Hurt (ver 3)"
Denegation error
Error in "Amen Omen"
Denegation error
Error in "Fallin (ver 2)"
Denegation error
Error in "Crazy For You"
Denegation error
Error in "Diamonds On The Inside"
Denegation error
Error in "Daydreamer (ver 4)"
Denegation error
Error in "Oo"
Denegation error
Error in "By Your Side"
Denegation error
Error in "Hurt (ver 2)"
Denegation error
Error in "Put Your Records On"
Denegation error
Error in "Sweet About Me (ver 3)"
Denegation error
Error in "Fuck Me Pumps"
Denegation error
Error in "Make You Feel My Love"
Denegation error
Error in "Back To Black"
Denegation error
Error in "If I Aint Got You (ver 2)"
Denegation error
Error in "Crazy (ver 2)"
Denegation error
Error in "You Know Im No Good (ver 4)"
Denegation error
Error i

KeyboardInterrupt: 

In [662]:
song_data.df

,url,name,genre,decade,chords,uuid
0,https://tabs.ultimate-guitar.com/tab/kodaline/...,All I Want,Rock,2010s,"['G', 'Em', 'C', 'D', 'G', 'G/B', 'C', 'Cm/Eb'...",f74f8254-3e6a-434a-b2b8-522fe9006939
1,https://tabs.ultimate-guitar.com/tab/hozier/ta...,Take Me To Church (ver 2),Rock,2010s,"['G', 'Em', 'C', 'D', 'G', 'G/B', 'C', 'Cm/Eb'...",c2398171-4a56-43dd-9c73-b7232e173595
2,https://tabs.ultimate-guitar.com/tab/imagine-d...,Radioactive,Rock,2010s,"['G', 'Em', 'C', 'D', 'G', 'G/B', 'C', 'Cm/Eb'...",5c76cdb6-ea7a-432c-a51a-319431142761
3,https://tabs.ultimate-guitar.com/tab/cage-the-...,Cigarette Daydreams,Rock,2010s,"['G', 'Em', 'C', 'D', 'G', 'G/B', 'C', 'Cm/Eb'...",5a17b07c-cbbb-4b8e-8bf9-85e1155c5b6c
4,https://tabs.ultimate-guitar.com/tab/a_great_b...,Say Something (ver 3),Rock,2010s,"['G', 'Em', 'C', 'D', 'G', 'G/B', 'C', 'Cm/Eb'...",79a27d58-200c-42de-b1c6-50b30fa98d5f
...,...,...,...,...,...,...
21655,https://tabs.ultimate-guitar.com/tab/sade/by_y...,By Your Side,Rhythm And Blues,2000s,NaN,NaN
21656,https://tabs.ultimate-guitar.com/tab/christina...,Hurt (ver 2),Rhythm And Blues,2000s,NaN,NaN
21657,https://tabs.ultimate-guitar.com/tab/corinne-b...,Put Your Records On,Rhythm And Blues,2000s,NaN,NaN
21658,https://tabs.ultimate-guitar.com/tab/gabriella...,Sweet About Me (ver 3),Rhythm And Blues,2000s,NaN,NaN


In [627]:
song_data.df[song_data.df['chords'].notnull()]

,url,name,genre,decade,chords,uuid
0,https://tabs.ultimate-guitar.com/tab/kodaline/...,All I Want,Rock,2010s,"['G', 'Em', 'C', 'D', 'G', 'G/B', 'C', 'Cm/Eb'...",f74f8254-3e6a-434a-b2b8-522fe9006939
1,https://tabs.ultimate-guitar.com/tab/hozier/ta...,Take Me To Church (ver 2),Rock,2010s,"['G', 'Em', 'C', 'D', 'G', 'G/B', 'C', 'Cm/Eb'...",c2398171-4a56-43dd-9c73-b7232e173595
2,https://tabs.ultimate-guitar.com/tab/imagine-d...,Radioactive,Rock,2010s,"['G', 'Em', 'C', 'D', 'G', 'G/B', 'C', 'Cm/Eb'...",5c76cdb6-ea7a-432c-a51a-319431142761
3,https://tabs.ultimate-guitar.com/tab/cage-the-...,Cigarette Daydreams,Rock,2010s,"['G', 'Em', 'C', 'D', 'G', 'G/B', 'C', 'Cm/Eb'...",5a17b07c-cbbb-4b8e-8bf9-85e1155c5b6c
4,https://tabs.ultimate-guitar.com/tab/a_great_b...,Say Something (ver 3),Rock,2010s,"['G', 'Em', 'C', 'D', 'G', 'G/B', 'C', 'Cm/Eb'...",79a27d58-200c-42de-b1c6-50b30fa98d5f
...,...,...,...,...,...,...
1995,https://tabs.ultimate-guitar.com/tab/ed-sheera...,Thinking Out Loud (ver 2),Folk,2010s,"['G', 'Em', 'C', 'D', 'G', 'G/B', 'C', 'Cm/Eb'...",0f06d373-31d9-417f-ac42-57fc29a433fa
1996,https://tabs.ultimate-guitar.com/tab/the_lumin...,Sleep On The Floor,Folk,2010s,"['G', 'Em', 'C', 'D', 'G', 'G/B', 'C', 'Cm/Eb'...",de60aa74-1592-4e8d-935f-34181a326401
1997,https://tabs.ultimate-guitar.com/tab/james-bay...,Let It Go (ver 3),Folk,2010s,"['G', 'Em', 'C', 'D', 'G', 'G/B', 'C', 'Cm/Eb'...",d556f4aa-539a-4f26-9946-a0a5c235ff1d
1998,https://tabs.ultimate-guitar.com/tab/ed-sheera...,Tenerife Sea (ver 4),Folk,2010s,"['G', 'Em', 'C', 'D', 'G', 'G/B', 'C', 'Cm/Eb'...",6a7388db-9342-4f54-b32d-9ade309183b1


In [628]:
song_data.save('../data/chords.csv')

In [629]:
df = song_data.df

In [653]:
len(df.groupby(['url', 'name', 'decade']))['url']

TypeError: 'int' object is not subscriptable

In [659]:
df.groupby(['url', 'name', 'decade'])['genre'].agg(lambda col: ''.join(col))

url                                                                               name                   decade
https://tabs.ultimate-guitar.com/tab/1136507                                      Clown                  2010s     Contemporary R&bContemporary R&bContemporary R...
https://tabs.ultimate-guitar.com/tab/1561735                                      Flor Pálida            2010s     World MusicWorld MusicWorld MusicWorld MusicWo...
https://tabs.ultimate-guitar.com/tab/1561736                                      Hipocresía             2010s     World MusicWorld MusicWorld MusicWorld MusicWo...
https://tabs.ultimate-guitar.com/tab/1624878                                      Llévame Contigo        2010s     World MusicWorld MusicWorld MusicWorld MusicWo...
https://tabs.ultimate-guitar.com/tab/1651168                                      Eres Tú                2010s     World MusicWorld MusicWorld MusicWorld MusicWo...
                                                               

In [664]:
df[df["name"] == "All I Want"]

,url,name,genre,decade,chords,uuid
0,https://tabs.ultimate-guitar.com/tab/kodaline/...,All I Want,Rock,2010s,"['G', 'Em', 'C', 'D', 'G', 'G/B', 'C', 'Cm/Eb'...",f74f8254-3e6a-434a-b2b8-522fe9006939
50,https://tabs.ultimate-guitar.com/tab/kodaline/...,All I Want,Rock,2010s,"['G', 'Em', 'C', 'D', 'G', 'G/B', 'C', 'Cm/Eb'...",f74f8254-3e6a-434a-b2b8-522fe9006939
100,https://tabs.ultimate-guitar.com/tab/kodaline/...,All I Want,Rock,2010s,"['G', 'Em', 'C', 'D', 'G', 'G/B', 'C', 'Cm/Eb'...",f74f8254-3e6a-434a-b2b8-522fe9006939
150,https://tabs.ultimate-guitar.com/tab/kodaline/...,All I Want,Rock,2010s,"['G', 'Em', 'C', 'D', 'G', 'G/B', 'C', 'Cm/Eb'...",f74f8254-3e6a-434a-b2b8-522fe9006939
200,https://tabs.ultimate-guitar.com/tab/kodaline/...,All I Want,Rock,2010s,"['G', 'Em', 'C', 'D', 'G', 'G/B', 'C', 'Cm/Eb'...",f74f8254-3e6a-434a-b2b8-522fe9006939
250,https://tabs.ultimate-guitar.com/tab/kodaline/...,All I Want,Rock,2010s,"['G', 'Em', 'C', 'D', 'G', 'G/B', 'C', 'Cm/Eb'...",f74f8254-3e6a-434a-b2b8-522fe9006939
300,https://tabs.ultimate-guitar.com/tab/kodaline/...,All I Want,Rock,2010s,"['G', 'Em', 'C', 'D', 'G', 'G/B', 'C', 'Cm/Eb'...",f74f8254-3e6a-434a-b2b8-522fe9006939
350,https://tabs.ultimate-guitar.com/tab/kodaline/...,All I Want,Rock,2010s,"['G', 'Em', 'C', 'D', 'G', 'G/B', 'C', 'Cm/Eb'...",f74f8254-3e6a-434a-b2b8-522fe9006939
400,https://tabs.ultimate-guitar.com/tab/kodaline/...,All I Want,Rock,2010s,"['G', 'Em', 'C', 'D', 'G', 'G/B', 'C', 'Cm/Eb'...",f74f8254-3e6a-434a-b2b8-522fe9006939
450,https://tabs.ultimate-guitar.com/tab/kodaline/...,All I Want,Rock,2010s,"['G', 'Em', 'C', 'D', 'G', 'G/B', 'C', 'Cm/Eb'...",f74f8254-3e6a-434a-b2b8-522fe9006939
